## Run MD Simulations

This notebook simulates multiple chains of NEAT1 with proteins (NONO, FUS, TDP-43)

### Load Libraries

In [ ]:
import sys
sys.path.append('../../Code/SimulationCode/')
import DataMangement as dm2p_ri
import Spatial_IC as ic
import CreateandRun_Spatial_Twoperiods_ri as car2p_ri
import matplotlib.pyplot as plt
sys.path.append('../../Code/AnalyzeData')
import AnalyzeData as ad 
import PlottingData as plot
import os

### Run Simulations

In [ ]:
#Read the interaction matrix
inputmatrix='InteractionMatrix_Example.csv'

In [ ]:
## Define experiment types
version='RNAand3Proteins_2P_ri' #don't change this
typefile='RNAand3Proteins' #don't change this 
experiment='Test' #name of folder to put in Data folder

#number of NEAT1 copies
Npolymer=5

#number of trajectories to run
num_trajectories=1

#create name from input file
name=inputmatrix.split('_')[1].split('.')[0]
for i in range(num_trajectories):
    subfolder=f'Interaction_{name}_NP_{Npolymer}'
    print(subfolder)

    #determine a random walk for RNA
    vproteins,vrna,seed,mdseed=ic.pre_calcs(Npolymer,L=45,vcalc='fixedbox',walk_type='multiplewalks',Nproteins=12000,spacing=24,Lbox=150)

    changes={'Npolymer':Npolymer}

    #parameters to name on trajectory files
    key_names=['L','Npolymer','Lbox','LE','P1Mep','vP1','spacing','seed']

    #create new input file based on interaction matrix, changes, and new parameters
    dictionary=dm2p_ri.create_input(inputmatrix,version,typefile,experiment,subfolder,changes,key_names,rnarna=False,change=False,rnarnatype=['Mep'], rnarnavalue=[1],testtype=['P1ep'], testvalue=[1])

    #run simulation
    car2p_ri.run_sim(dictionary['JobInputName'])


### Analyze Data

In [ ]:
folder='Data/Experiments/Test/Interaction_Example_NP_5'
LC=True #Analyze the largest cluster
num_files=15 #number of files to analyze from the end of the simulation

#Calculate the average values of the data
ad.run_avg(folder, calckey=['all'],num_files=num_files,LC=LC)

#Plot the data
plot_LC=True #Plot the largest cluster
dataanalysisfolder=folder+'/DataAnalysis/'
experimenttypename= 'LE15_NP5_Test_LC' #Name for plots
plot.plot_data(dataanalysisfolder, experimenttypename, plot_LC,save=False)

### Plot density profile with standard deviation 

In [ ]:
import pickle
#pull average and all trajectory files ran for density profile
densityfolder=dataanalysisfolder+'/Density_LC/'
for file in os.listdir(densityfolder):
    if file.endswith('_avg'):
        avg=densityfolder+file
    else:
        alltrajs= densityfolder+file

#plot density profile with standard deviation
plot.plot_den_alltrajs_density(alltrajs,avg)